### QuickStart: Compare runs, Choose a model, and deploy it to the REST API ###

In this quickstart, you will: 
 - Run a hyperparameter sweep on  a training script
 - Compare the results of the runs in the MLFLOW UI
 - Choose the best run and register it as a model
 - Deploy the model to the REST API 
 - Build a container image suitable for the deployment to the cloud platform

In [65]:
import keras
import numpy as np
import pandas as pd
from hyperopt import STATUS_OK, Trials, fmin, hp,tpe
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import mlflow
from mlflow.models import infer_signature


In [66]:
# Load the data

data=pd.read_csv(
   "https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv ",
    sep=";",
)
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [67]:
## Split the data into training,validation and st sets 

train, test = train_test_split(data, test_size=0.2, random_state=42)

# Define the model



In [68]:
train_x = train.drop(["quality"], axis=1).values
train_y = train["quality"].values.ravel()

## Test Dataset
test_x = test.drop(["quality"], axis=1).values
test_y = test["quality"].values.ravel()

## Split the training data into training and validation sets
train_x, valid_x, train_y, valid_y = train_test_split(train_x, train_y, test_size=0.2, random_state=42)
signature = infer_signature(train_x,train_y)

# Define the search space



In [69]:
### Ann Model

from numpy import shape


def train_model(params,epochs,train_x,train_y,valid_x,valid_y,test_x,test_y):

    ##Define model architechture
    mean = np.mean(train_x, axis=0)
    var = np.var(train_x, axis=0)   ## Both the mean and the variance is used to perform normalization

    model = keras.Sequential(
        [
            keras.Input([train_x.shape[1]]),  ## Number of data in train_x
            keras.layers.Normalization(mean = mean, variance = var),  ## Performing Normalization
            keras.layers.Dense(64, activation = 'relu'),  ## Declaing the number of neurons 
            keras.layers.Dense(1) 
        ]
    )


    ##Compile the model that we made above 
    model.compile(optimizer = keras.optimizers.SGD(
    learning_rate=params["lr"], momentum = params['momentum']  ## Model will be using differnt learning rate that we will mention in the prams
    ),
                 loss = "mean_squared_error",
                 metrics = [keras.metrics.RootMeanSquaredError()])
    
    
     ## Train the ANN model with lr and momentum params with MLFLOW tracking
    with mlflow.start_run(nested=True):
        model.fit(train_x,train_y,validation_data=(valid_x,valid_y),
                  epochs=epochs,
                  batch_size=64)
        
        ## Evaluate the model (Best model, run)
        eval_result=model.evaluate(valid_x,valid_y,batch_size=64)

        eval_rmse=eval_result[1]

        ## Log the parameters and results
        mlflow.log_params(params)
        mlflow.log_metric("eval_rmse",eval_rmse)

        ## log the model (we used the tensorflow for the keras)

        mlflow.tensorflow.log_model(model,"model",signature=signature)

        return {"loss": eval_rmse, "status": STATUS_OK, "model": model}

In [70]:
def objective(params):
    # MLflow will track the parameters and results for each run
    result = train_model(
        params,
        epochs=3,
        train_x=train_x,
        train_y=train_y,
        valid_x=valid_x,
        valid_y=valid_y,
        test_x=test_x,
        test_y=test_y,
    )
    return result

In [71]:
space={
    "lr":hp.loguniform("lr",np.log(1e-5),np.log(1e-1)),  ## The learning rate should be in between 10^5 and 10^1 only
    "momentum":hp.uniform("momentum",0.0,1.0)

}

In [72]:
mlflow.set_experiment("wine-quality")
with mlflow.start_run():
    # Conduct the hyperparameter search using Hyperopt
    trials=Trials()  ## This is going to perform the hyperparameter tuning
    best=fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=4,
        trials=trials
    )

    # Fetch the details of the best run
    best_run = sorted(trials.results, key=lambda x: x["loss"])[0]

    # Log the best parameters, loss, and model
    mlflow.log_params(best)
    mlflow.log_metric("eval_rmse", best_run["loss"])
    mlflow.tensorflow.log_model(best_run["model"], "model", signature=signature)

    # Print out the best parameters and corresponding loss
    print(f"Best parameters: {best}")
    print(f"Best eval rmse: {best_run['loss']}")


  0%|          | 0/4 [00:00<?, ?trial/s, best loss=?]

Epoch 1/3                                            

 1/49 ━━━━━━━━━━━━━━━━━━━━ 55s 1s/step - loss: 38.2101 - root_mean_squared_error: 6.1814
27/49 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 28.5707 - root_mean_squared_error: 5.3269
46/49 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 24.0352 - root_mean_squared_error: 4.8567
49/49 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 23.2649 - root_mean_squared_error: 4.7718 - val_loss: 3.7813 - val_root_mean_squared_error: 1.9446

Epoch 2/3                                            

 1/49 ━━━━━━━━━━━━━━━━━━━━ 2s 42ms/step - loss: 3.7659 - root_mean_squared_error: 1.9406
25/49 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3.0272 - root_mean_squared_error: 1.7376 
46/49 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2.7569 - root_mean_squared_error: 1.6567
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 2.7202 - root_mean_squared_error: 1.6454 - val_loss: 2.1575 - val_root_mean_squared_error: 1.4689

Epoch 3/3                                            

 1/49 ━━━